In [1]:
import pandas as pd
import argparse as args
import numpy as np


# Height formatting:

#### Reading the data:

In [2]:
height = pd.DataFrame(pd.read_csv("~/alzheimersproject/1_raw_data/height_yengo", 
                delim_whitespace=True, header=0, na_values='NA'))

height

,SNPID,RSID,CHR,POS,EFFECT_ALLELE,OTHER_ALLELE,EFFECT_ALLELE_FREQ,BETA,SE,P,N
0,1:566875:C:T,rs2185539,1,566875,T,C,0.002800,-0.046316,0.03930,0.238128,537968.0
1,1:728951:C:T,rs11240767,1,728951,T,C,0.000356,0.167358,0.12600,0.185025,85591.0
2,1:734462:A:G,rs12564807,1,734462,A,G,0.893000,0.004657,0.01100,0.672866,112953.0
3,1:752721:A:G,rs3131972,1,752721,G,A,0.840000,0.000544,0.00284,0.848110,615932.0
4,1:754182:A:G,rs3131969,1,754182,G,A,0.865000,0.001333,0.00185,0.470389,1100634.0
...,...,...,...,...,...,...,...,...,...,...,...
1373015,22:51188282:A:G,rs28420908,22,51188282,A,G,0.001400,-0.017847,0.01730,0.301195,919162.0
1373016,22:51193445:A:G,rs6010077,22,51193445,A,G,0.000103,0.293986,0.27700,0.287716,63112.0
1373017,22:51211392:C:T,rs3888396,22,51211392,C,T,0.103000,0.003989,0.00317,0.208649,705911.0
1373018,22:51212875:A:C,rs2238837,22,51212875,C,A,0.337000,-0.002627,0.00206,0.201820,859105.0


From the table above, Z is the only missing column.

#### Adding an Z column:

In [10]:
%%bash

awk -v OFS='\t' 'BEGIN { print "SNP", "CHR", "BP", "A1", "A2", "Z", "P", "N", "FREQ", "BETA", "SE" } NR>2 { print $2, $3, $4, $5, $6, $12 = ($8/$9), $10, int($11), $7, $8, $9 }' ~/alzheimersproject/1_raw_data/height_yengo > height_all_cols


#### Removing potential non-rsIDs in the SNP column:

In [11]:
%%bash

awk -v OFS='\t' 'NR == 1 || $1 ~ "rs"' height_all_cols > height_onlyrs

#### Removing possible duplicates in the file:

In [17]:
%%bash

awk '!seen[$1]++' height_onlyrs > height_onlyrs_uniq

#### Checking the file:

In [13]:
height = pd.DataFrame(pd.read_csv("~/alzheimersproject/2_formatting/height_onlyrs_uniq", 
                delim_whitespace=True, header=0, na_values='NA'))

height

,SNP,CHR,BP,A1,A2,Z,P,N,FREQ,BETA,SE
0,rs11240767,1,728951,T,C,1.328240,0.185025,85591.0,0.000356,0.167358,0.12600
1,rs12564807,1,734462,A,G,0.423355,0.672866,112953.0,0.893000,0.004657,0.01100
2,rs3131972,1,752721,G,A,0.191581,0.848110,615932.0,0.840000,0.000544,0.00284
3,rs3131969,1,754182,G,A,0.720600,0.470389,1100634.0,0.865000,0.001333,0.00185
4,rs3131967,1,754334,C,T,0.772535,0.440485,1095682.0,0.866000,0.001429,0.00185
...,...,...,...,...,...,...,...,...,...,...,...
1372442,rs28420908,22,51188282,A,G,-1.031600,0.301195,919162.0,0.001400,-0.017847,0.01730
1372443,rs6010077,22,51193445,A,G,1.061320,0.287716,63112.0,0.000103,0.293986,0.27700
1372444,rs3888396,22,51211392,C,T,1.258240,0.208649,705911.0,0.103000,0.003989,0.00317
1372445,rs2238837,22,51212875,C,A,-1.275440,0.201820,859105.0,0.337000,-0.002627,0.00206


#### Checking for NaN, NA, inf:

In [8]:
# 1. Checking for NaN:
print(height.isnull().sum())

print('\n------------------------------\n')

# 2. Checking for NA:
print(height.isna().sum())

print('\n------------------------------\n')

# 3. Checking for inf:
print((height.isin([np.inf, -np.inf])).sum())

SNP       0
CHR       0
BP        0
A1        0
A2        0
Z         0
P         0
N         0
FREQ    410
BETA      0
SE      410
dtype: int64

------------------------------

SNP       0
CHR       0
BP        0
A1        0
A2        0
Z         0
P         0
N         0
FREQ    410
BETA      0
SE      410
dtype: int64

------------------------------

SNP     0
CHR     0
BP      0
A1      0
A2      0
Z       0
P       0
N       0
FREQ    0
BETA    0
SE      0
dtype: int64


In [23]:
%%bash

# Checking for P=0:
awk -v OFS='\t' 'NR == 1 || $7 == "0.0" || $7 == "0"' height_onlyrs_uniq | head


SNP	CHR	BP	A1	A2	Z	P	N	FREQ	BETA	SE
rs17029613	1	6592800	C	T	0		0	0.000	NA	7998
rs2232452	1	6663745	A	G	0		0	0.000	NA	11890
rs2274973	1	11851424	T	C	0		0	0.000	NA	11890
rs5227	1	11918817	A	C	0		0	0.000	NA	7998
rs11569926	1	12188123	T	C	0		0	0.000	NA	4804
rs6676236	1	20940489	A	G	0		0	0.000	NA	9797
rs11580192	1	22378555	A	G	0		0	0.000	NA	2436
rs665	1	24181041	T	C	0		0	0.000	NA	7998
rs16837597	1	28828359	C	T	0		0	0.000	NA	8944


#### Deleting variants where P=0:
This is done as Z, freq, beta, se, and N is also zero here.

In [1]:
%%bash

awk -v OFS='\t' 'NR == 1 || $7 != "0"' height_onlyrs_uniq > height_formatted